# Transformers

Selections from the [Chapter 16](https://github.com/ageron/handson-ml3/blob/main/16_nlp_with_rnns_and_attention.ipynb) notebook from the Scikit-learn book. Much like the textbook author, I gave up trying to make it work with Keras 3.

In [1]:
# Connect google drive for persistence
from google.colab import drive
from pathlib import Path

drive.mount("/content/drive")
model_root = Path("/content/drive/MyDrive/SavedModels/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import sys
IS_COLAB = "google.colab" in sys.modules
if IS_COLAB:
    import os
    os.environ["TF_USE_LEGACY_KERAS"] = "1"
    import tf_keras

from packaging import version
import tensorflow as tf

assert version.parse(tf.__version__) >= version.parse("2.8.0")

In [4]:
if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. Neural nets can be very slow without a GPU.")
    if "google.colab" in sys.modules:
        print("Go to Runtime > Change runtime and select a GPU hardware "
              "accelerator.")
    if "kaggle_secrets" in sys.modules:
        print("Go to Settings > Accelerator and select GPU.")


We'll continue on with the English-Spanish translation task, so let's re-download and prepare the data.

In [8]:
from pathlib import Path
# Even loading the data has to change for keras 2/3
url = "https://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip"
path = tf.keras.utils.get_file("spa-eng.zip", origin=url, cache_dir="datasets",
                               extract=True)
text = (Path(path).with_name("spa-eng") / "spa.txt").read_text()

2638744/2638744 [==============================] - 0s 0us/step


In [9]:
import numpy as np

text = text.replace("¡", "").replace("¿", "")
pairs = [line.split("\t") for line in text.splitlines()]
np.random.seed(42)  # extra code – ensures reproducibility on CPU
np.random.shuffle(pairs)
sentences_en, sentences_es = zip(*pairs)  # separates the pairs into 2 lists

In [10]:
vocab_size = 1000
max_length = 50
batch_size = 32

text_vec_layer_en = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode = "int",
    output_sequence_length=max_length,
    pad_to_max_tokens=True,
)
text_vec_layer_es = tf.keras.layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode = "int",
    output_sequence_length=max_length,
    pad_to_max_tokens=True,
)
text_vec_layer_en.adapt(sentences_en)
text_vec_layer_es.adapt([f"startofseq {s} endofseq" for s in sentences_es])

In [11]:
X_train = tf.constant(sentences_en[:100_000])
X_valid = tf.constant(sentences_en[100_000:])
X_train_dec = tf.constant([f"startofseq {s}" for s in sentences_es[:100_000]])
X_valid_dec = tf.constant([f"startofseq {s}" for s in sentences_es[100_000:]])
Y_train = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[:100_000]])
Y_valid = text_vec_layer_es([f"{s} endofseq" for s in sentences_es[100_000:]])

## RNN with Attention
We'll define a bidirectional LSTM model, but this time add an `Attention` layer.

The following cell is the same as in the vanilla LSTM examples.

In [12]:
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
encoder = tf.keras.layers.Bidirectional(
    tf.keras.layers.LSTM(256, return_sequences=True, return_state=True))

encoder_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)
decoder_inputs = tf.keras.layers.Input(shape=(1,), dtype=tf.string)

# fairly arbitrary size for the word embeddings
# You could probably sub in pre-trained embeddings here
embed_size = 128
encoder_input_ids = text_vec_layer_en(encoder_inputs)
decoder_input_ids = text_vec_layer_es(decoder_inputs)
encoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
decoder_embedding_layer = tf.keras.layers.Embedding(vocab_size, embed_size,
                                                    mask_zero=True)
encoder_embeddings = encoder_embedding_layer(encoder_input_ids)
decoder_embeddings = decoder_embedding_layer(decoder_input_ids)

encoder_outputs, *encoder_state = encoder(encoder_embeddings)
encoder_state = [tf.keras.layers.Concatenate(axis=-1)([encoder_state[0], encoder_state[2]]),  # short-term
                 tf.keras.layers.Concatenate(axis=-1)([encoder_state[1], encoder_state[3]])]  # long-term

decoder = tf.keras.layers.LSTM(512, return_sequences=True)
decoder_outputs = decoder(decoder_embeddings, initial_state=encoder_state)

Instead of just connecting the encoder and decoder directly, we'll put an `Attention` layer in the middle.

In [13]:
attention_layer = tf.keras.layers.Attention()
# query = decoder_outputs, value = encoder_outputs
attention_outputs = attention_layer([decoder_outputs, encoder_outputs])
output_layer = tf.keras.layers.Dense(vocab_size, activation="softmax")
Y_proba = output_layer(attention_outputs)

In [14]:
# trying to figure out shape mismatch error, no luck
for tensor in [decoder_outputs, encoder_outputs, attention_outputs]:
    print(tensor.shape)

(None, 50, 512)
(None, 50, 512)
(None, 50, 512)


**Warning**: the following cell will take a while to run (possibly a couple hours if you are not using a GPU).

In [15]:
attn_path = model_root / "attention_model.keras"
if attn_path.exists():
    attention_model = tf.keras.models.load_model(attn_path)
else:
    attention_model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs], outputs=[Y_proba])
    attention_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
                metrics=["accuracy"])
    attention_model.fit((X_train, X_train_dec), Y_train, epochs=10,
            validation_data=((X_valid, X_valid_dec), Y_valid))
    attention_model.save(attn_path)

attention_model.summary()

Epoch 1/10
3125/3125 [==============================] - 159s 43ms/step - loss: 2.8322 - accuracy: 0.4561 - val_loss: 1.8771 - val_accuracy: 0.5818
Epoch 2/10
3125/3125 [==============================] - 117s 37ms/step - loss: 1.6187 - accuracy: 0.6291 - val_loss: 1.4885 - val_accuracy: 0.6554
Epoch 3/10
3125/3125 [==============================] - 90s 29ms/step - loss: 1.3476 - accuracy: 0.6810 - val_loss: 1.3511 - val_accuracy: 0.6833
Epoch 4/10
3125/3125 [==============================] - 85s 27ms/step - loss: 1.2019 - accuracy: 0.7074 - val_loss: 1.2917 - val_accuracy: 0.6940
Epoch 5/10
3125/3125 [==============================] - 87s 28ms/step - loss: 1.0936 - accuracy: 0.7280 - val_loss: 1.2588 - val_accuracy: 0.7013
Epoch 6/10
3125/3125 [==============================] - 85s 27ms/step - loss: 1.0046 - accuracy: 0.7453 - val_loss: 1.2538 - val_accuracy: 0.7021
Epoch 7/10
3125/3125 [==============================] - 79s 25ms/step - loss: 0.9287 - accuracy: 0.7607 - val_loss: 1.2599

In [16]:
# encode/decode one word at a time until we predict endofseq
def translate(model, sentence_en):
    translation = ""
    for word_idx in range(max_length):
        X = tf.constant([sentence_en])  # encoder input
        X_dec = tf.constant(["startofseq " + translation])  # decoder input
        y_proba = model.predict((X, X_dec))[0, word_idx]  # last token's probas
        predicted_word_id = np.argmax(y_proba)
        predicted_word = text_vec_layer_es.get_vocabulary()[predicted_word_id]
        if predicted_word == "endofseq":
            break
        translation += " " + predicted_word
    return translation.strip()

In [17]:
translate(attention_model, "Where is the beach?")

1/1 [==============================] - 0s 21ms/step


'dónde está la playa'

In [18]:
translate(attention_model, "I love to go to the beach, do you know where I can find it?")

1/1 [==============================] - 0s 22ms/step


'me encanta ir a la playa hace [UNK] dónde puedo encontrar'

## Attention Is All You Need: The Transformer Architecture
### Positional encodings

In [19]:
max_length = 50  # max length in the whole training set
embed_size = 128
tf.random.set_seed(42)  # extra code – ensures reproducibility on CPU
pos_embed_layer = tf.keras.layers.Embedding(max_length, embed_size)
batch_max_len_enc = encoder_embeddings.shape[1]
encoder_in = encoder_embeddings + pos_embed_layer(tf.range(batch_max_len_enc))
batch_max_len_dec = decoder_embeddings.shape[1]
decoder_in = decoder_embeddings + pos_embed_layer(tf.range(batch_max_len_dec))

Alternatively, we can use fixed, non-trainable positional encodings:

In [20]:
class PositionalEncoding(tf.keras.layers.Layer):
    def __init__(self, max_length, embed_size, dtype=tf.float32, **kwargs):
        super().__init__(dtype=dtype, **kwargs)
        assert embed_size % 2 == 0, "embed_size must be even"
        p, i = np.meshgrid(np.arange(max_length),
                           2 * np.arange(embed_size // 2))
        pos_emb = np.empty((1, max_length, embed_size))
        pos_emb[0, :, ::2] = np.sin(p / 10_000 ** (i / embed_size)).T
        pos_emb[0, :, 1::2] = np.cos(p / 10_000 ** (i / embed_size)).T
        self.pos_encodings = tf.constant(pos_emb.astype(self.dtype))
        self.supports_masking = True

    def call(self, inputs):
        batch_max_length = tf.shape(inputs)[1]
        return inputs + self.pos_encodings[:, :batch_max_length]

In [21]:
pos_embed_layer = PositionalEncoding(max_length, embed_size)
encoder_in = pos_embed_layer(encoder_embeddings)
decoder_in = pos_embed_layer(decoder_embeddings)

### Multi-Head Attention

In [22]:
N = 2  # instead of 6
num_heads = 8
dropout_rate = 0.1
n_units = 128  # for the first Dense layer in each Feed Forward block
encoder_pad_mask = tf.math.not_equal(encoder_input_ids, 0)[:, tf.newaxis]
Z = encoder_in
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.Dropout(dropout_rate)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

In [23]:
decoder_pad_mask = tf.math.not_equal(decoder_input_ids, 0)[:, tf.newaxis]
causal_mask = tf.linalg.band_part(  # creates a lower triangular matrix
    tf.ones((batch_max_len_dec, batch_max_len_dec), tf.bool), -1, 0)

In [24]:
encoder_outputs = Z  # let's save the encoder's final outputs
Z = decoder_in  # the decoder starts with its own inputs
for _ in range(N):
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=Z, attention_mask=causal_mask & decoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    attn_layer = tf.keras.layers.MultiHeadAttention(
        num_heads=num_heads, key_dim=embed_size, dropout=dropout_rate)
    Z = attn_layer(Z, value=encoder_outputs, attention_mask=encoder_pad_mask)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))
    skip = Z
    Z = tf.keras.layers.Dense(n_units, activation="relu")(Z)
    Z = tf.keras.layers.Dense(embed_size)(Z)
    Z = tf.keras.layers.LayerNormalization()(tf.keras.layers.Add()([Z, skip]))

**Warning**: the following cell will take a while to run (possibly 2 or 3 hours if you are not using a GPU).

In [26]:
transformer_path = model_root / "transformer.keras"
if transformer_path.exists():
    transformer_model = tf.keras.models.load_model(transformer_path)
else:
    Y_proba = tf.keras.layers.Dense(vocab_size, activation="softmax")(Z)
    transformer_model = tf.keras.Model(inputs=[encoder_inputs, decoder_inputs],
                        outputs=[Y_proba])
    transformer_model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam",
                metrics=["accuracy"])
    transformer_model.fit((X_train, X_train_dec), Y_train, epochs=10,
            validation_data=((X_valid, X_valid_dec), Y_valid))
    transformer_model.save(transformer_path)

transformer_model.summary()

Epoch 1/10
3125/3125 [==============================] - 187s 50ms/step - loss: 1.9635 - accuracy: 0.5802 - val_loss: 1.4632 - val_accuracy: 0.6581
Epoch 2/10
3125/3125 [==============================] - 138s 44ms/step - loss: 1.4262 - accuracy: 0.6606 - val_loss: 1.3463 - val_accuracy: 0.6789
Epoch 3/10
3125/3125 [==============================] - 149s 48ms/step - loss: 1.3153 - accuracy: 0.6807 - val_loss: 1.2603 - val_accuracy: 0.6997
Epoch 4/10
3125/3125 [==============================] - 140s 45ms/step - loss: 1.2471 - accuracy: 0.6925 - val_loss: 1.2129 - val_accuracy: 0.7041
Epoch 5/10
3125/3125 [==============================] - 148s 47ms/step - loss: 1.1960 - accuracy: 0.7019 - val_loss: 1.2013 - val_accuracy: 0.7052
Epoch 6/10
3125/3125 [==============================] - 140s 45ms/step - loss: 1.1548 - accuracy: 0.7096 - val_loss: 1.1529 - val_accuracy: 0.7160
Epoch 7/10
3125/3125 [==============================] - 150s 48ms/step - loss: 1.1222 - accuracy: 0.7151 - val_loss: 1

In [30]:
translate(transformer_model, "I like soccer and also going to the beach")

1/1 [==============================] - 0s 83ms/step


'me gusta el fútbol y iba a la playa'

# HuggingFace

Install the Transformers and Datasets libraries if we're running on Colab:

In [31]:
import sys
if "google.colab" in sys.modules:
    %pip install -q -U transformers
    %pip install -q -U datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 64.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.6 MB/s eta 0:00:00


In [32]:
from transformers import pipeline

classifier = pipeline("sentiment-analysis")  # many other tasks are available
result = classifier("The actors were very convincing.")

No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

Device set to use cuda:0


In [33]:
result

[{'label': 'POSITIVE', 'score': 0.9998071789741516}]

Models can be very biased. For example, it may like or dislike some countries depending on the data it was trained on, and how it is used, so use it with care:

In [34]:
classifier(["I am from India.", "I am from Iraq."])

[{'label': 'POSITIVE', 'score': 0.9896161556243896},
 {'label': 'NEGATIVE', 'score': 0.9811071157455444}]

In [35]:
model_name = "huggingface/distilbert-base-uncased-finetuned-mnli"
classifier_mnli = pipeline("text-classification", model=model_name)
classifier_mnli("She loves me. [SEP] She loves me not.")

config.json:   0%|          | 0.00/729 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/58.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cuda:0


[{'label': 'contradiction', 'score': 0.9790192246437073}]

In [36]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModelForSequenceClassification.from_pretrained(model_name)

All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [37]:
token_ids = tokenizer(["I like soccer. [SEP] We all love soccer!",
                       "Joe lived for a very long time. [SEP] Joe is old."],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [38]:
token_ids = tokenizer([("I like soccer.", "We all love soccer!"),
                       ("Joe lived for a very long time.", "Joe is old.")],
                      padding=True, return_tensors="tf")
token_ids

{'input_ids': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[ 101, 1045, 2066, 4715, 1012,  102, 2057, 2035, 2293, 4715,  999,
         102,    0,    0,    0],
       [ 101, 3533, 2973, 2005, 1037, 2200, 2146, 2051, 1012,  102, 3533,
        2003, 2214, 1012,  102]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(2, 15), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0],
       [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [39]:
outputs = model(token_ids)
outputs

TFSequenceClassifierOutput(loss=None, logits=<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[-2.1123817 ,  1.17868   ,  1.4100995 ],
       [-0.01478346,  1.0962477 , -0.99199575]], dtype=float32)>, hidden_states=None, attentions=None)

In [40]:
Y_probas = tf.keras.activations.softmax(outputs.logits)
Y_probas

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[0.01619703, 0.43523633, 0.54856664],
       [0.2265597 , 0.6881726 , 0.08526774]], dtype=float32)>

In [41]:
Y_pred = tf.argmax(Y_probas, axis=1)
Y_pred  # 0 = contradiction, 1 = entailment, 2 = neutral

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([2, 1])>

## Chat Models
Note: to access the Llama model, I needed to sign up for Hugging Face, request access/sell all my info to Meta, and configure the `HF_TOKEN` in Colab. Then, my session crashed after using all available RAM.

An open (and much smaller) model to try is `model="distilbert/distilgpt2"`.

In [42]:
# bluey = pipeline("text-generation", model="meta-llama/Llama-2-7b-chat-hf")
bluey = pipeline("text-generation", model="distilbert/distilgpt2")

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cuda:0


In [49]:
# test it out!
prompt = "Help, I'm stuck in a GPU"
print(bluey(prompt, pad_token_id=50256, max_new_tokens=200)[0]['generated_text'])

Help, I'm stuck in a GPU-assisted "brain-synchronization machine" (FAT) that is not currently supported and there is absolutely no way to make anything work with FAT hardware, as I am not running the FAT in my native C++ environment. I've run both the Visual Studio 5.2 "graphics library" and Visual Studio 6.0. This is an example of the machine's performance (and the use of Visual Studio 8.1), but not to the point where I want to use Visual Studio 7.2 for applications.

For now, these are some ways that I can easily work with FAT hardware using Windows:
The Visual Studio 7.2 GUI application to create a new one. You should now be familiar with Windows on Visual Studio 7. If you don't have Visual Studio 7.2 setup from Windows 10.1, then start from the Windows Insider Hub. You can use your Windows 10.1/WMI-enabled machine and click the "
